In [1]:
all_chars = "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [2]:
chars = [i for i in all_chars]

In [3]:
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [5]:
# print(encode("hii there"))
# print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
from typing import List, Optional, Tuple

import hydra
import lightning as L
import pyrootutils
import torch
from lightning import Callback, LightningDataModule, LightningModule, Trainer
from lightning.pytorch.loggers import Logger
from omegaconf import DictConfig

pyrootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)
# from src import utils

# log = utils.get_pylogger(__name__)


/home/rampfire/anaconda3/envs/pytorch_model_training/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PosixPath('/home/rampfire/ERA__V_1/vision_meets_nlp')

In [8]:
from hydra import compose, initialize
from omegaconf import OmegaConf
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="../../configs", job_name="test_app")
cfg = compose(config_name="train.yaml", overrides=["experiment=gpt/gpt.yaml"])
print(OmegaConf.to_yaml(cfg))

/tmp/ipykernel_8195/1090264794.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../../configs", job_name="test_app")


task_name: train
tags:
- llm
- nanogpt
train: true
test: false
compile: false
ckpt_path: null
seed: 12345
data:
  _target_: src.data.tinyshakespearedatamodule.TinyShakespeareDataloader
  data_dir: ${paths.data_dir}
  batch_size: 256
  train_val_test_split:
  - 55000
  - 5000
  - 10000
  num_workers: 4
  pin_memory: false
model:
  _target_: src.models.bigram_language_pl.BigramLanguageModelLitModule
  vocab_size: 65
  optimizer:
    _target_: torch.optim.Adam
    _partial_: true
    lr: 0.0001
    weight_decay: 0.0001
  scheduler: null
callbacks:
  model_checkpoint:
    _target_: lightning.pytorch.callbacks.ModelCheckpoint
    dirpath: ${paths.output_dir}/checkpoints
    filename: epoch_{epoch:03d}
    monitor: val/acc
    verbose: false
    save_last: true
    save_top_k: 1
    mode: max
    auto_insert_metric_name: false
    save_weights_only: false
    every_n_train_steps: null
    train_time_interval: null
    every_n_epochs: null
    save_on_train_epoch_end: null
  early_stopping:
 

In [9]:
model: LightningModule = hydra.utils.instantiate(cfg.model)

In [10]:
checkpoint = torch.load("/home/rampfire/ERA__V_1/vision_meets_nlp/logs/train/runs/last_gpt.ckpt",map_location=torch.device('cpu'))
# model.load_state_dict(checkpoint['model_state_dict'])

In [11]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [12]:
# idx = torch.zeros((1, 1), dtype=torch.long)
# idx.shape

torch.Size([1, 1])

In [13]:
# encoded_sample = encode("hii there")
# encoded_sample

[46, 47, 47, 1, 58, 46, 43, 56, 43]

In [14]:
# data = torch.tensor([encoded_sample], dtype=torch.long)
# data,data.shape

(tensor([[46, 47, 47,  1, 58, 46, 43, 56, 43]]), torch.Size([1, 9]))

In [15]:
# model.model_.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)
# print(decode(model.model_.generate(data, max_new_tokens=500)[0].tolist()))

hii thereleeoulbre hnouspouat ce t tse tonsp w tobe O the thee w tosewoff y ht m. w ap tshe the offf-ng sow s---- y are bu, y the busche totre mp Ce

 medne y to----he itw tne a ppon y tp che me the tstoupeng cre offuse che, te se



the y tithese ty titre tme ine itshnse ce ate




 kipp ssp thawe itoe ido-che t-- ip w the taf tone twonsp me a tw ve tsp sp-- ch tonome w he tw
 tofonobShe fon ship me tche a ing a o--shife it ofibee w mimishe m? mme a je mp
 thi h swe atod e sp y ta-ffThe w sp te y tofof 


In [18]:
def generate_response(input_text):
    encoded_input_text = encode(input_text)
    data = torch.tensor([encoded_input_text], dtype=torch.long)
    return decode(model.model_.generate(data, max_new_tokens=500)[0].tolist())


In [19]:
generate_response("hi there")

"hi there spelldeiuserererioul, t.\n\n\n\n ta che I pousp the tse wed tof tome tone ishe w tshe ine tthe e---p the\n\nsp p she one the buse tref--\n\n'sspsh; thinse affe te me t t w my ho sp e the thee d- itw\n\n\n\n\nithe;\n\n yonese eathane the os sptoust me tod te s----\n\n\nstease twe a the tofoos tone se timse spse to\n\nare tse he the mpoure t he te t se a ite csp im; ts.\nbe e a the t y wow twic itsse sse eHe ine tshe y ne are w w- chosp-Se\n one me se i-- spppe tofe te t\n\n\n\n\n\nshele\nipoufe ne we y thit. che sp me tot t"

In [21]:
import gradio as gr

In [23]:
input_text_box = gr.Textbox()
output_text_box = gr.Textbox()

In [24]:
app  = gr.Interface(generate_response,input_text_box,output_text_box)

In [25]:
app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
